In [1]:
import csv
import numpy as np
import pandas as pd


In [2]:
TextForElmo = pd.read_csv("textforEmbedding.csv")


C:\Users\Kasutaja\Anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
TextSubset = TextForElmo[1000:1500] 

In [19]:
oneString = ''.join(map(str,  TextSubset["text"].tolist()))

In [21]:
import nltk

In [22]:
sentence_tokenizer = nltk.data.load('tokenizers/punkt/estonian.pickle')

In [24]:
listOfSentences = sentence_tokenizer.tokenize(oneString)

In [33]:
myTextDF = pd.DataFrame({"transcript":listOfSentences})

In [39]:
myTextDF[3500:3550]

,transcript
3500,"Kirjeldus: Par.käe IV sõrme prox,falange ristifraktuur."
3501,"Kuupäev: Kood ja nimetus: 79200 Ühe mähisega uuring (kuni 3 tööd) keskmise väljaga MRT-l Põhjendus: mrt peaajust;\nkognitiivne düsfktsioon, I tüüpi diabeet Kirjeldus: KESKMISE <ANONYM id=""4"" type=""per"" morph=""_S_ sg kom""/> MRT-UURING PEAAJUST: Loc, Sag.T1 SE, Ax T2 frFSE, Ax DWI 1000b, Ax T2 FLAIR,Ax SE T1, Cor T2 FLAIR\n\nLEID: \nAju külgvatsakesed, III ja IV vatsake on normaalse kuju ja laiusega, deformatsioonita ja dislokatsioonita."
3502,Ekstratserebraalsed liikvoriruumid on normaalsed.
3503,Suuraju poolkerades subkortikaalsel üksikud paari millimeetrised T2 intensiivsed kolded.
3504,"Muus osas on ajukude koldemuutusteta, DW muutusteta."
3505,Järeldus: Üksikud kolded subkortikaalsel- tõenäoliselt isheemilis-gliootilised.
3506,"Kuupäev: 15.01.2013 16:16 Kood ja nimetus: 79202 Ühe mähisega uuring (6-7 tööd) keskmise väljaga MRT-l Põhjendus: mrt peaajust;\nkognitiivne düsfktsioon, I tüüpi diabeet Kirjeldus: KESKMISE <ANONYM id=""5"" type=""per"" morph=""_S_ sg kom""/> MRT-UURING PEAAJUST: Loc, Sag.T1 SE, Ax T2 frFSE, Ax DWI 1000b, Ax T2 FLAIR,Ax SE T1, Cor T2 FLAIR\n\nLEID: \nAju külgvatsakesed, III ja IV vatsake on normaalse kuju ja laiusega, deformatsioonita ja dislokatsioonita."
3507,Ekstratserebraalsed liikvoriruumid on normaalsed.
3508,Suuraju poolkerades subkortikaalsel üksikud paari millimeetrised T2 intensiivsed kolded.
3509,"Muus osas on ajukude koldemuutusteta, DW muutusteta."


In [40]:
train_df = myTextDF.sample(frac=0.8, random_state=100)
test_df = myTextDF[~myTextDF.index.isin(train_df.index)]


In [47]:
# train_df['transcript'] = train_df['transcript'] + " ."

train_df['transcript'][200:210]

2160                                                           <ANONYM id="33" type="per" morph="_A_ pl p"/> väikesi bullasid.
1782                                                                                                                  Peaseis.
1877    Kuupäev: 11.07.2013 12:56 Kood ja nimetus: 7984 Kompuutertomograafia-angiograafia ühest piirkonnast Põhjendus: tuumor?
426                                              Pankrease mõõtmed on normi piires, struktuur, kontrasteerumine iseärasusteta.
6963                                                                       Aspirin, Plavix, Norvasc, Betaloc-Zoc, Simvastatin.
244                                                                                         Mõõtmetelt maks suurenenud ei ole.
3853                                                                      Kontrastainena süstitud  Visipaque 270 - 110 ml i/v.
4694                                     Esineb hüpokinees inferoseptaalses seinas basaalses segmendis ja infer

In [134]:
import os
if not os.path.exists("swb/train"):
    os.makedirs("swb/train")
 
for i in range(0,train_df.shape[0],6):
    text = "\n".join(train_df['transcript'][i:i+6].tolist())
    fp = open("swb/train/"+str(i)+".txt","w",encoding="UTF-8")
    fp.write(text)
    fp.close()

#Prepare Validation Data


In [136]:
test_df['transcript'] = test_df['transcript'] + " ."
if not os.path.exists("swb/dev"):
    os.makedirs("swb/dev")
 
for i in range(0,test_df.shape[0],6):
    text = "\n".join(test_df['transcript'][i:i+6].tolist())
    fp = open("swb/dev/"+str(i)+".txt","w",encoding="UTF-8")
    fp.write(text)
    fp.close()

C:\Users\Kasutaja\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#Prepare vocab file

In [137]:
from collections import Counter
vocab = " ".join(train_df['transcript'].tolist())
vocab_words = vocab.split(" ")
print("Number of tokens in Training data = ",len(vocab_words))
dictionary = Counter(vocab_words)
print("Size of Vocab",len(dictionary))
sorted_vocab = ["&lt;S&gt;","&lt;/S&gt;","&lt;UNK&gt;"]
sorted_vocab.extend([pair[0] for pair in dictionary.most_common()])
 
sorted_text = "\n".join(sorted_vocab)
fp = open("swb/vocab.txt","w",encoding="UTF-8")
fp.write(sorted_text)
fp.close()

Number of tokens in Training data =  86912
Size of Vocab 8797


In [50]:
!pip install os

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
